In [1]:
import openai
import requests
import psycopg2
from psycopg2.extras import execute_values

In [2]:
# Configure OpenAI API
openai.api_key = ""
# PostgreSQL connection parameters
DATABASE_CONFIG = {
    "dbname": "news_db",
    "user": "news_user",
    "password": "news_password",
    "host": "localhost",  # Or the IP/hostname of your database
    "port": 5432
}

In [3]:
# Define a function to query news articles
def query_news():
    url = 'https://newsapi.org/v2/top-headlines'
    params = {
        'country': 'us',
        'apiKey': 'c05f140b4860486cba5633ca73c2f014',
    }
    response = requests.get(url, params=params)
    if response.status_code == 200:
        data = response.json()
        return data.get('articles', [])
    else:
        print(f"Error fetching news: {response.status_code}")
        return []

# Define a function to generate embeddings
def generate_embedding(text):
    import time
    start = time.time()
    response = openai.Embedding.create(
        input=text,
        model="text-embedding-ada-002"
    )
    elapsed = time.time() - start
    print(elapsed)
    return response['data'][0]['embedding']

# Define a function to insert articles into the database
def store_articles_in_db(articles):
    connection = psycopg2.connect(**DATABASE_CONFIG)
    cursor = connection.cursor()

    # Ensure the table exists
    cursor.execute("""
        CREATE TABLE IF NOT EXISTS articles (
            id SERIAL PRIMARY KEY,
            content TEXT,
            embedding VECTOR(1536)
        );
    """)

    # Prepare data for insertion
    values = []
    for article in articles:
        content = article.get('content')
        if content:
            embedding = generate_embedding(content)
            values.append((content, embedding))

    # Insert data into the table
    if values:
        insert_query = "INSERT INTO articles (content, embedding) VALUES %s"
        execute_values(cursor, insert_query, values, template=None, page_size=100)

    connection.commit()
    cursor.close()
    connection.close()

In [4]:
# Main execution
if __name__ == "__main__":
    # Fetch articles
    articles = query_news()
    
    # Store articles in the database
    store_articles_in_db(articles)

    print("Articles and embeddings successfully stored in the database.")

0.5552151203155518
0.6219415664672852
0.4624636173248291
0.6938731670379639
0.5000715255737305
0.39806604385375977
0.3244030475616455
0.31168699264526367
0.1899263858795166
0.5937438011169434
0.6490416526794434
0.22485065460205078
0.4251258373260498
0.20440101623535156
0.4131011962890625
0.4811592102050781
0.8073930740356445
Articles and embeddings successfully stored in the database.
